# DESAFIO STONE:

## PROPOSTA:

Access the PostgreSQL database using the following credentials:

    host: db-stone.cjepwwjnksng.us-east-1.rds.amazonaws.com
    port: 5432
    database name: postgres
    user: read_only_user
    password: banking123


The database contains credit card transactional data in 4 tables:

    customers
    cards
    transactions
    frauds

Develop code to extract 2 kinds of datasets and export them to csv files. The required datasets are the following:

    1) The number of transactions and the total value purchased of each credit card grouped by card number and card family.
    2) All the customer ids that have "Diamond" segment and made at least 40 transactions.
   

Extras:

The following extra tasks are not required, but can give your solution bonus points.

    3) The table frauds shows all the transaction ids that were proven to be fraudulent. Analyze the data to find a correlation between the fraudulent transactions and the other features of the dataset. Explain your results.

    4) Develop a Dockerfile and/or a docker-compose file to automate your data processing application.
_______________________________________________________________________________________________________________________________

## SOLUÇÃO:

### - Importando as bibliotecas básicas e me conectando ao banco de dados:

Conectando à base de dados ao python...
Seguindo o tutorial  de https://www.postgresqltutorial.com/postgresql-python/connect/ :

1 - É preciso importar os módulos csv e psycopg2 (possui os métodos para fazer a conexão):

In [1]:
import csv
import psycopg2

In [2]:
import math  #será usado futuramente...

2 - É preciso criar uma variável da conexão:

In [3]:
con = psycopg2.connect(
                        host='db-stone.cjepwwjnksng.us-east-1.rds.amazonaws.com',
                        port=5432,
                        database='postgres',
                        user='read_only_user',
                        password='banking123')

3 - Também devemos criar uma variável para o "cursor", que fará a comunicação com o banco de dados.

In [4]:
cur = con.cursor()

### - Conhecendo as tabelas e seus relacionamentos:

In [5]:
#tupla com os nomes das tabelas:
nomes_tabelas=('customers','cards','transactions','frauds')

In [6]:
#conhecendo as colunas de cada tabela:
for t in nomes_tabelas:
    query_header = "SELECT column_name FROM information_schema.columns WHERE table_name = '"+t+"'"
    cur.execute(query_header)
    header = cur.fetchall()
    
    query_exemplo = "SELECT * FROM "+t+" LIMIT 1"
    cur.execute(query_exemplo)
    exemplo = cur.fetchone()
    print("%s:\n%s\n%s\n"% (t,header,exemplo))

customers:
[('id',), ('age',), ('segment',), ('vintage_group',)]
('CC25034', 35, 'Diamond', 'VG1')

cards:
[('card_number',), ('card_family',), ('credit_limit',), ('customer_id',)]
('8638-5407-3631-8196', 'Premium', 530000, 'CC67088')

transactions:
[('id',), ('card_number',), ('transaction_date',), ('value',), ('segment',)]
('CTID28830551', '1629-9566-3285-2123', datetime.date(2016, 4, 24), 23649, 'SEG25')

frauds:
[('transaction_id',), ('fraud_flag',)]
('CTID50558449', True)



In [7]:
#Decidi checar se havia diferença entre as colunas "segment", que aparecem nas tabelas "customers" e "transactions"
cur.execute('SELECT DISTINCT segment FROM customers')
row = cur.fetchall()
print("segments de 'customers':\n%s\n"%row)
cur.execute('SELECT DISTINCT segment FROM transactions')
row = cur.fetchall()
print("segments de 'transactions':\n%s" %row)

segments de 'customers':
[('Platinum',), ('Gold',), ('Diamond',)]

segments de 'transactions':
[('SEG16',), ('SEG20',), ('SEG19',), ('SEG23',), ('SEG17',), ('SEG15',), ('SEG24',), ('SEG22',), ('SEG18',), ('SEG11',), ('SEG13',), ('SEG12',), ('SEG14',), ('SEG25',), ('SEG21',)]


In [8]:
#Checando se as tabelas Frauds e Transactions tem o mesmo número de linhas...
cur.execute("SELECT COUNT(*) FROM transactions")
query1 = cur.fetchall()
cur.execute("SELECT COUNT(*) FROM frauds")
query2 = cur.fetchall()
lista = [query1[0][0],query2[0][0]]
print(lista)

[10000, 109]


In [9]:
#Confirmando que a tabela frauds só carrega as transações com fraudes...
cur.execute("SELECT DISTINCT fraud_flag FROM frauds")
tipos_de_fraude = cur.fetchall()
print(tipos_de_fraude)

[(True,)]


### Resolvendo os itens 1 e 2

In [10]:
#Resolução do item 1:
cur.execute("SELECT c.card_number,c.card_family,COUNT(t.id), SUM(t.value) FROM transactions t INNER JOIN cards c ON t.card_number = c.card_number GROUP BY c.card_number,c.card_family ORDER BY SUM(t.value) DESC")
item_1 = cur.fetchall()

#Salvando item A em .csv no arquivo "Item_1.csv"
saida = open("Item_1.csv",'w',newline = '')
escrever = csv.writer(saida)
escrever.writerow(("Card Number","Card Family","Transaction Number","Total Value Purchased"))
for i in item_1:
    escrever.writerow(i)
saida.close()

In [11]:
#Resolvendo o item 2:
cur.execute("SELECT cus.id,cus.segment,COUNT(tran.id) FROM transactions tran INNER JOIN cards car ON tran.card_number = car.card_number INNER JOIN customers cus ON car.customer_id = cus.id WHERE cus.segment = 'Diamond' GROUP BY cus.id,cus.segment HAVING COUNT(tran.id)>39 ORDER BY COUNT(tran.id) DESC")
item_2 = cur.fetchall()

#Salvando item 2 em .csv no arquivo "ItemB.csv"
saida = open("Item_2.csv",'w',newline = '')
escrever = csv.writer(saida)
escrever.writerow(("Custumer ID","Segment","Total Number of Transactions"))
for i in item_2:
    escrever.writerow(i)
saida.close()

## Extras:

### - Item 3:

Para resolver o item 3, a minha estratégia foi:
- Criar uma tabela que traga todas as transações, a indicação se a transação é fraudulenta e as variáveis que eu quero testar.
- Testar a dependência de fraud_flag em cada uma dessas variáveis, por usando o método do Qui-Quadrado.

    (Me baseei nos conhecimentos adquiridos em uma disciplina de simulação e nos tutoriais: "A Gentle Introduction to the Chi-Squared Test for Machine Learning" e "How to Choose a Feature Selection Method For Machine Learning" do site https://machinelearningmastery.com)

In [12]:
#Pegando a Tabela Desejada:
cur.execute("SELECT tran.id,transaction_date,value,tran.segment,fraud_flag,card_family,credit_limit,age,cus.segment,vintage_group FROM transactions tran INNER JOIN cards car ON tran.card_number = car.card_number INNER JOIN customers cus ON car.customer_id = cus.id LEFT JOIN frauds f on tran.id=f.transaction_id")
transacoes_expandida = cur.fetchall()
print("Exemplo de linha da tabela:\ntran.id, transaction_date, value, tran.segment, fraud_flag, card_family, credit_limit, age, cus.segment, vintage_group")
print(transacoes_expandida[0])

Exemplo de linha da tabela:
tran.id, transaction_date, value, tran.segment, fraud_flag, card_family, credit_limit, age, cus.segment, vintage_group
('CTID28830551', datetime.date(2016, 4, 24), 23649, 'SEG25', None, 'Platinum', 194000, 23, 'Gold', 'VG3')


In [13]:
cur.close()
con.close()

In [14]:
# Transformando a Tabela Extraída em uma Lista de Listas(que podem ser modificadas mais facilmente):
lista = []
for i in transacoes_expandida: #Aqui eu decidi transformar os valores de "fraud_flag" em 1(é fraude) e 0(não é fraude). 
    k = list(i)
    if k[4]==True:
        k[4]=1
    else:
        k[4]=0
    lista.append(k)
    
conta_fraude = 0 #Fiz essa contagem para verificar se eu extrai corretamente todos as linhas.
conta_todo = 0   #O certo seria contar 109 fraudes e 10000 transações
for i in lista:
    conta_todo += 1
    if i[4]==1:
        conta_fraude += 1
print(conta_fraude,conta_todo)

109 10000


#### Testes Qui-Quadradro:

Nesta etapa, farei testes qui-quadrado da coluna "fraud_flag" com as demais colunas, uma de cada vez.
Esse teste estatístico permite checar a relação de dependência entre duas variáveis categóricas.

No nosso caso, as variáveis categóricas são:

    fraud_flag, tran.segment, card_family, cus.segment, vintage_group

Também temos três variáveis numéricas, que são:

    value, credit_limit, age

Irei começar tratando as variáveis numéricas, que podem ser transformadas em categorias pela regra de Sturges...

In [15]:
# Usando a regra de Sturges para definir o número de categorias (k)...
# k = 1+ 3.3*log(n); onde n é o tamanho da amostra(número de transações fraudulentas, calculado anteriormente)
tamanho_amostra = conta_fraude
k = math.ceil(1+3.3*math.log(tamanho_amostra,10))
print("Tamanho da amostra: %d\nNúmero de Categorias K: %d"%(tamanho_amostra,k))

Tamanho da amostra: 109
Número de Categorias K: 8


In [16]:
#Vamos começar avaliando a relação entre "frauds" e "values"...

#É preciso saber a amplitude do dados "values", ou seja: qual é a diferença entre o maior e o menor valor?
lvalue = []
for i in lista:
    lvalue.append(i[2])

values_amplitude = max(lvalue) - min(lvalue)
print(values_amplitude)

# para a classificação, os valores serão classificados em "k" intervalos diferentes de tamanho "h":

h = values_amplitude/k
print(h)

# A lista que é criada abaixo, mostra os limites superiores dos intervalos de classificação:
ini_class_values = []
for i in range(1,(k+1)):
    ini_class_values.append(int(i*h + min(lvalue)))
print(ini_class_values)    

49892
6236.5
[6339, 12576, 18812, 25049, 31285, 37522, 43758, 49995]


In [17]:
#O próximo passo é criar a "Tabela Contingência" que irá contar as ocorências em cada classificação:

tabela_fraudesValues = [[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
for i in lista:
    #descubro a posição da contagem de value:
    pos = min(int((i[2]-min(lvalue))/h),(k-1))
    tabela_fraudesValues[i[4]][pos] += 1
print(tabela_fraudesValues)

[[1272, 1255, 1269, 1229, 1251, 1231, 1198, 1186], [7, 15, 14, 9, 14, 28, 9, 13]]


In [18]:
#Importo as bibliotecas que onde o método Qui-Quadrado, já foi desenvolvido
from scipy.stats import chi2_contingency
from scipy.stats import chi2

In [19]:
#A função a seguir retorna a estatística calculada, o "p-valor", os graus de liberdade e a tabela de frequências esperada:
estat, p, gdl, esperada = chi2_contingency(tabela_fraudesValues)
print('gdl=%d' % gdl)
print(esperada)
# interpretando o teste estatistico:
prob = 0.95
critica = chi2.ppf(prob, gdl)
print('probabilidade=%.3f, crítica=%.3f, estatísca=%.3f' % (prob, critica, estat))
if abs(estat) >= critica:
    print('Variáveis Dependentes (rejeita H0)')
else:
    print('Variáveis Independentes (falha em rejeitar H0)')
# interpretando o p-valor
alpha = 1.0 - prob
print('nível de significância=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Variáveis Dependentes (rejeita H0)')
else:
    print('Variáveis Independentes (falha em rejeitar H0)')

gdl=7
[[1265.0589 1256.157  1269.0153 1224.5058 1251.2115 1245.2769 1193.8437
  1185.9309]
 [  13.9411   13.843    13.9847   13.4942   13.7885   13.7231   13.1563
    13.0691]]
probabilidade=0.950, crítica=14.067, estatísca=21.453
Variáveis Dependentes (rejeita H0)
nível de significância=0.050, p=0.003
Variáveis Dependentes (rejeita H0)


# ------------------------------------------------------------------------------------------
De forma análoga, as tabelas contingência de "frauds" com "credit limits" e "age" serão criadas e postas à prova:

In [20]:
#Criação da tabela contingência Fraudes x Limite de Crédito
l_CredLim = []
for i in lista:
    l_CredLim.append(i[6]) #coluna de índice 6 na lista 

CredLim_amplitude = max(l_CredLim) - min(l_CredLim)

h = CredLim_amplitude/k
print(h)
ini_class_CredLim = []
for i in range(1,(k+1)):
    ini_class_CredLim.append(int(i*h + min(l_CredLim)))
print(ini_class_CredLim)

112125.0
[114125, 226250, 338375, 450500, 562625, 674750, 786875, 899000]


In [21]:
tabela_fraudesCredLim = [[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
for i in lista:
    #descubro a posição da contagem de "credit limits":
    pos = min(int((i[6]-min(l_CredLim))/h),(k-1))
    tabela_fraudesCredLim[i[4]][pos] += 1
print(tabela_fraudesCredLim)

[[4423, 1969, 644, 564, 393, 676, 634, 588], [44, 26, 10, 9, 5, 4, 6, 5]]


In [22]:
estat, p, gdl, esperada = chi2_contingency(tabela_fraudesCredLim)
print('gdl=%d' % gdl)
print(esperada)
# interpretando o teste estatistico:
prob = 0.95
critica = chi2.ppf(prob, gdl)
print('probabilidade=%.3f, crítica=%.3f, estatísca=%.3f' % (prob, critica, estat))
if abs(estat) >= critica:
    print('Variáveis Dependentes (rejeita H0)')
else:
    print('Variáveis Independentes (falha em rejeitar H0)')
# interpretando o p-valor
alpha = 1.0 - prob
print('nível de significância=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Variáveis Dependentes (rejeita H0)')
else:
    print('Variáveis Independentes (falha em rejeitar H0)')

gdl=7
[[4.4183097e+03 1.9732545e+03 6.4687140e+02 5.6675430e+02 3.9366180e+02
  6.7258800e+02 6.3302400e+02 5.8653630e+02]
 [4.8690300e+01 2.1745500e+01 7.1286000e+00 6.2457000e+00 4.3382000e+00
  7.4120000e+00 6.9760000e+00 6.4637000e+00]]
probabilidade=0.950, crítica=14.067, estatísca=5.859
Variáveis Independentes (falha em rejeitar H0)
nível de significância=0.050, p=0.556
Variáveis Independentes (falha em rejeitar H0)


In [23]:
#Criação da tabela contingência Fraudes x Idades
l_Ages = []
for i in lista:
    l_Ages.append(i[7]) #coluna de índice 7 na lista 

Ages_amplitude = max(l_Ages) - min(l_Ages)

h = Ages_amplitude/k
print(h)
ini_class_Ages = []
for i in range(1,(k+1)):
    ini_class_Ages.append(int(i*h + min(l_Ages)))
print(ini_class_Ages)

tabela_fraudesAges = [[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
for i in lista:
    #descubro a posição da contagem de "ages":
    pos = min(int((i[7]-min(l_Ages))/h),(k-1))
    tabela_fraudesAges[i[4]][pos] += 1
print(tabela_fraudesAges)

3.75
[23, 27, 31, 35, 38, 42, 46, 50]
[[991, 1357, 1276, 1013, 1242, 1400, 1129, 1483], [12, 13, 17, 11, 13, 23, 6, 14]]


In [24]:
estat, p, gdl, esperada = chi2_contingency(tabela_fraudesAges)
print('gdl=%d' % gdl)
print(esperada)
# interpretando o teste estatistico:
prob = 0.95
critica = chi2.ppf(prob, gdl)
print('probabilidade=%.3f, crítica=%.3f, estatísca=%.3f' % (prob, critica, estat))
if abs(estat) >= critica:
    print('Variáveis Dependentes (rejeita H0)')
else:
    print('Variáveis Independentes (falha em rejeitar H0)')
# interpretando o p-valor
alpha = 1.0 - prob
print('nível de significância=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Variáveis Dependentes (rejeita H0)')
else:
    print('Variáveis Independentes (falha em rejeitar H0)')

gdl=7
[[ 992.0673 1355.067  1278.9063 1012.8384 1241.3205 1407.4893 1122.6285
  1480.6827]
 [  10.9327   14.933    14.0937   11.1616   13.6795   15.5107   12.3715
    16.3173]]
probabilidade=0.950, crítica=14.067, estatísca=8.307
Variáveis Independentes (falha em rejeitar H0)
nível de significância=0.050, p=0.306
Variáveis Independentes (falha em rejeitar H0)


### Conclusão:
#### - Até o momento, pelo método Qui-Quadrado, apenas o valor da transação mostrou alguma ligação com as fraudes...

**Falta fazer procedimento similar com as demais variáveis. Por serem categóricas, elas serão testadas de forma mais direta...